# Importing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee
import geemap
import fiona
import time
import json
from pathlib import Path
import datetime
import numpy as np
from utils.gee import export_sar, export_opt, export_maskcloud, export_opt_l1
import yaml

# Authentication

In [3]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


# Site Selection

In [28]:
site = 3
stie_cfg = f'site_{site}.yaml'

In [33]:
with open(stie_cfg, 'r') as file:
    cfg = yaml.safe_load(file)

In [34]:
cfg

{'xcenter': -37.023,
 'ycenter': -7.497,
 'xmin': -37.5,
 'ymin': -7.947,
 'xmax': -36.637,
 'ymax': -7.023,
 'date': ['2019-10-24', '2020-10-10', '2021-10-13'],
 'n_sar_avg_images': 12,
 'before_days': 31,
 'opt_images_dates': [['2019-11-02', '2019-11-03'],
  ['2019-10-23', '2019-10-24'],
  ['2019-10-28', '2019-10-29']],
 'tiles_params': {'lines': 4,
  'columns': 4,
  'train_tiles': [1, 2, 4, 7, 8, 9, 13, 15]},
 'original_data': {'opt_imgs': {'train': ['2019-10-28-2019-10-29.tif',
    '2019-10-18-2019-10-19.tif',
    '2019-11-07-2019-11-08.tif',
    '2020-10-07-2020-10-08.tif',
    '2020-10-17-2020-10-18.tif',
    '2020-10-22-2020-10-23.tif'],
   'test': ['2020-10-07-2020-10-08.tif',
    '2020-10-17-2020-10-18.tif',
    '2020-10-22-2020-10-23.tif',
    '2021-10-12-2021-10-13.tif',
    '2021-10-17-2021-10-18.tif',
    '2021-10-02-2021-10-03.tif']},
  'sar_imgs': {'train': ['2019-09-23-2019-10-24.tif',
    '2019-10-25-2019-11-25.tif',
    '2019-11-26-2019-12-27.tif',
    '2019-12-28-202

# Reading Location data

In [9]:
roi = ee.Geometry.Rectangle([cfg['xmin'], cfg['ymin'], cfg['xmax'], cfg['ymax']])

In [35]:
map_view = geemap.Map(center=(cfg['ycenter'], cfg['xcenter']), zoom=8)
map_view


Map(center=[-7.497, -37.023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [36]:
#location = geemap.shp_to_ee(str(location_shape))
map_view.addLayer(roi, {}, 'Location')

# Dates

In [12]:
before_days = cfg['before_days']
d0 = datetime.datetime.strptime(cfg['date'][0], "%Y-%m-%d")
d1 = datetime.datetime.strptime(cfg['date'][1], "%Y-%m-%d")
d2 = datetime.datetime.strptime(cfg['date'][2], "%Y-%m-%d")
dt1 = (d1-d0).days + before_days
dt2 = (d2-d1).days + before_days

In [13]:
start_date = d0 - datetime.timedelta(days=before_days)
end_date = d2 + datetime.timedelta(days=before_days)
print(f'Analyzed Period: {start_date.date()}:{end_date.date()}')


Analyzed Period: 2019-09-23:2021-11-13


In [15]:
opt_vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 2048
}

sar_vizParams = {
  'bands': ['VV'],
  'min': 0,
  'max': 1
}

cloud_vizParams = {
  'bands': ['MSK_CLDPRB'],
  'min': 0,
  'max': 100
}

# SAR Data

## Average Data

In [16]:
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(start_date.date()), str(end_date.date())).filterBounds(roi)
ds = np.arange(dt1)
img_collections = []
split_ds = np.array_split(ds, cfg['n_sar_avg_images'])
for i, dsi in enumerate(split_ds):
    di0 = d0 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[0]))
    di1 = d0 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[-1]))
    img_col = sar_collection.filterDate(str(di0.date()), str(di1.date()))
    img_col = img_col.filter(ee.Filter.eq('platform_number', 'A'))
    img = img_col.mean().clip(roi)
    #print(f'{di0.date()}-{di1.date()}-delta:{(di1-di0).days}|{len(img_col.getInfo()["features"])} images')
    print(f'{di0.date()}-{di1.date()}.tif')
    #map_view.addLayer(img, sar_vizParams, f't1-{i:02d}')
    export_sar(img, roi, f'{di0.date()}-{di1.date()}')
    
    

avg_2019-09-23-2019-10-24.tif
avg_2019-10-25-2019-11-25.tif
avg_2019-11-26-2019-12-27.tif
avg_2019-12-28-2020-01-28.tif
avg_2020-01-29-2020-02-29.tif
avg_2020-03-01-2020-04-01.tif
avg_2020-04-02-2020-05-03.tif
avg_2020-05-04-2020-06-04.tif
avg_2020-06-05-2020-07-06.tif
avg_2020-07-07-2020-08-07.tif
avg_2020-08-08-2020-09-08.tif
avg_2020-09-09-2020-10-09.tif


In [17]:
ds = np.arange(dt2)
img_collections = []
split_ds = np.array_split(ds,  cfg['n_sar_avg_images'])
for i, dsi in enumerate(split_ds):
    di0 = d1 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[0]))
    di1 = d1 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[-1]))
    img_col = sar_collection.filterDate(str(di0.date()), str(di1.date()))
    img_col = img_col.filter(ee.Filter.eq('platform_number', 'A'))
    img = img_col.mean().clip(roi)
    #print(f'{di0.date()}-{di1.date()}-delta:{(di1-di0).days}|{len(img_col.getInfo()["features"])} images')
    print(f'{di0.date()}-{di1.date()}.tif')
    #map_view.addLayer(img, sar_vizParams, f't1-{i:02d}')
    export_sar(img, roi, f'{di0.date()}-{di1.date()}')

avg_2020-09-09-2020-10-12.tif
avg_2020-10-13-2020-11-15.tif
avg_2020-11-16-2020-12-19.tif
avg_2020-12-20-2021-01-21.tif
avg_2021-01-22-2021-02-23.tif
avg_2021-02-24-2021-03-28.tif
avg_2021-03-29-2021-04-30.tif
avg_2021-05-01-2021-06-02.tif
avg_2021-06-03-2021-07-05.tif
avg_2021-07-06-2021-08-07.tif
avg_2021-08-08-2021-09-09.tif
avg_2021-09-10-2021-10-12.tif


## Multiple Images

In [18]:
date_0, date_1 = d0 - datetime.timedelta(days=12), d1 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
skip_idx = []
count = 0
for idx, cycle in enumerate(cycles):
    if idx in skip_idx:
        continue
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.mosaic()
    data_check = img.mask().reduceRegion(
        ee.Reducer.min(), 
        roi,
        scale = 10,
        maxPixels = 1e9).getInfo()
    #if not 0 in data_check.values():
    if data_check['angle'] == 1:
        count+=1
        img = img.clip(roi)
        map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: {dates[0].date()}-{dates[-1].date()}.tif')
        #export_sar(img, roi, f'{dates[0].date()}-{dates[-1].date()}')
print(count)

122
2019-10-12---2020-10-22
31
0: 2019-10-16-2019-10-21.tif
1: 2019-10-28-2019-11-05.tif
2: 2019-11-09-2019-11-17.tif
3: 2019-11-21-2019-11-26.tif
4: 2019-12-03-2019-12-08.tif
5: 2019-12-15-2019-12-20.tif
6: 2019-12-27-2020-01-01.tif
7: 2020-01-08-2020-01-13.tif
8: 2020-01-20-2020-01-25.tif
9: 2020-02-01-2020-02-06.tif
10: 2020-02-13-2020-02-18.tif
11: 2020-02-25-2020-03-01.tif
12: 2020-03-08-2020-03-13.tif
13: 2020-03-20-2020-03-25.tif
14: 2020-04-01-2020-04-06.tif
15: 2020-04-13-2020-04-18.tif
16: 2020-04-25-2020-04-30.tif
17: 2020-05-07-2020-05-12.tif
19: 2020-05-31-2020-06-05.tif
20: 2020-06-12-2020-06-17.tif
21: 2020-06-24-2020-06-29.tif
22: 2020-07-06-2020-07-11.tif
23: 2020-07-18-2020-07-23.tif
24: 2020-07-30-2020-08-04.tif
25: 2020-08-11-2020-08-16.tif
26: 2020-08-23-2020-08-28.tif
27: 2020-09-04-2020-09-09.tif
28: 2020-09-16-2020-09-21.tif
30: 2020-10-10-2020-10-15.tif
29


In [19]:
date_0, date_1 = d1 - datetime.timedelta(days=12), d2 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
skip_idx = []
count = 0
for idx, cycle in enumerate(cycles):
    if idx in skip_idx:
        continue
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.mosaic()
    data_check = img.mask().reduceRegion(
        ee.Reducer.min(), 
        roi,
        scale = 10,
        maxPixels = 1e9).getInfo()
    #if not 0 in data_check.values():
    if data_check['angle'] == 1:
        count+=1
        img = img.clip(roi)
        #map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: {dates[0].date()}-{dates[-1].date()}.tif')
        #export_sar(img, roi, f'{dates[0].date()}-{dates[-1].date()}')
print(count)

128
2020-09-28---2021-10-25
33
1: 2020-10-10-2020-10-15.tif
2: 2020-10-22-2020-10-27.tif
3: 2020-11-03-2020-11-08.tif
4: 2020-11-15-2020-11-20.tif
5: 2020-11-27-2020-12-02.tif
6: 2020-12-09-2020-12-14.tif
7: 2020-12-21-2020-12-26.tif
8: 2021-01-02-2021-01-07.tif
9: 2021-01-14-2021-01-19.tif
10: 2021-01-26-2021-01-31.tif
11: 2021-02-07-2021-02-12.tif
12: 2021-02-19-2021-02-24.tif
13: 2021-03-03-2021-03-08.tif
14: 2021-03-15-2021-03-20.tif
15: 2021-03-27-2021-04-01.tif
16: 2021-04-08-2021-04-13.tif
17: 2021-04-20-2021-04-25.tif
18: 2021-05-02-2021-05-07.tif
19: 2021-05-14-2021-05-19.tif
20: 2021-05-26-2021-05-31.tif
21: 2021-06-07-2021-06-12.tif
22: 2021-06-19-2021-06-24.tif
23: 2021-07-01-2021-07-06.tif
24: 2021-07-13-2021-07-18.tif
25: 2021-07-25-2021-07-30.tif
27: 2021-08-18-2021-08-23.tif
28: 2021-08-30-2021-09-04.tif
29: 2021-09-11-2021-09-16.tif
30: 2021-09-23-2021-09-28.tif
31: 2021-10-05-2021-10-10.tif
32: 2021-10-17-2021-10-22.tif
31


## Log data 2 dates

In [149]:
date_0, date_1 = d0 - datetime.timedelta(days=12), d1 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
cycles = [cycle for i, cycle in enumerate(cycles) if i == 0 or i == (len(cycles)-2) ]
for idx, cycle in enumerate(cycles):
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.median()
    data_check = img.mask().reduceRegion(
        ee.Reducer.mean(), 
        roi,
        scale = 10,
        maxPixels = 1e9).getInfo()
    #if not 0 in data_check.values():
    map_view.addLayer(img.clip(roi), sar_vizParams, f't2-{str(cycle)}')
    if data_check['VH'] >= 0.999:
        img = img.clip(roi).float()
        map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: log_{dates[0].date()}-{dates[-1].date()}.tif')
        export_sar(img, roi, f'log_{dates[0].date()}-{dates[-1].date()}')

130
2019-07-12---2020-08-07
33
0: log_2019-07-14-2019-07-21.tif
1: log_2020-07-20-2020-07-27.tif


In [152]:
date_0, date_1 = d1 - datetime.timedelta(days=12), d2 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
cycles = [cycle for i, cycle in enumerate(cycles) if i == 1 or i == (len(cycles)-2) ]
for idx, cycle in enumerate(cycles):
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.median()
    data_check = img.mask().reduceRegion(
        ee.Reducer.mean(), 
        roi,
        scale = 10,
        maxPixels = 1e9).getInfo()
    #if not 0 in data_check.values():
    map_view.addLayer(img.clip(roi), sar_vizParams, f't2-{str(cycle)}')
    if data_check['VH'] >= 0.999:
        img = img.clip(roi).float()
        map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: log_{dates[0].date()}-{dates[-1].date()}.tif')
        export_sar(img, roi, f'log_{dates[0].date()}-{dates[-1].date()}')

132
2020-07-14---2021-08-10
34
0: log_2020-07-20-2020-07-27.tif
1: log_2021-07-27-2021-08-03.tif


# Optical Data (Level 2A)

In [37]:
opt_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(start_date, end_date)
opt_collection = opt_collection.filterBounds(roi)
opt_info = opt_collection.getInfo()

In [25]:
for info in opt_info['features']:
    img_date = datetime.datetime.strptime(info['properties']['DATATAKE_IDENTIFIER'][5:13], '%Y%m%d')
    img_lct = info['properties']['PRODUCT_ID']
    cloud = info['properties']['CLOUDY_PIXEL_PERCENTAGE']
    print(img_date, img_lct, cloud)
    #break

2019-09-28 00:00:00 S2B_MSIL2A_20190928T125309_N0213_R052_T24MXS_20190928T150147 0.006417
2019-09-28 00:00:00 S2B_MSIL2A_20190928T125309_N0213_R052_T24MXT_20190928T150147 0.005551
2019-09-28 00:00:00 S2B_MSIL2A_20190928T125309_N0213_R052_T24MYS_20190928T150147 0.044841
2019-09-28 00:00:00 S2B_MSIL2A_20190928T125309_N0213_R052_T24MYT_20190928T150147 0.026682
2019-10-03 00:00:00 S2A_MSIL2A_20191003T125311_N0213_R052_T24MXS_20191003T150217 0.168304
2019-10-03 00:00:00 S2A_MSIL2A_20191003T125311_N0213_R052_T24MXT_20191003T150217 0.773239
2019-10-03 00:00:00 S2A_MSIL2A_20191003T125311_N0213_R052_T24MYS_20191003T150217 11.906125
2019-10-03 00:00:00 S2A_MSIL2A_20191003T125311_N0213_R052_T24MYT_20191003T150217 8.331555
2019-10-08 00:00:00 S2B_MSIL2A_20191008T125309_N0213_R052_T24MXS_20191008T140453 0.089313
2019-10-08 00:00:00 S2B_MSIL2A_20191008T125309_N0213_R052_T24MXT_20191008T140453 0.013533
2019-10-08 00:00:00 S2B_MSIL2A_20191008T125309_N0213_R052_T24MYS_20191008T140453 0.71305
2019-10-08

In [38]:
for date_i in cfg['opt_images_dates']:
    opt_d0 = date_i[0]
    opt_d1 = date_i[1]
    opt_collection_i = opt_collection.filterDate(opt_d0, opt_d1)
    opt_mosaic_i = opt_collection_i.mosaic()
    #export_opt(opt_mosaic_i, roi, f'{opt_d0}-{opt_d1}')
    #export_maskcloud(opt_mosaic_i, roi, f'cloud_{opt_d0}-{opt_d1}')
    map_view.addLayer(opt_mosaic_i.clip(roi), opt_vizParams, f'opt-{opt_d0}-{opt_d1}')
    print(f'{opt_d0}-{opt_d1}.tif')
    #print(f'{len(opt_collection_i.getInfo()["features"])} images')

2019-11-02-2019-11-03.tif
2019-10-23-2019-10-24.tif
2019-10-28-2019-10-29.tif


# Optical Data (Level 1C)

In [29]:
opt_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(start_date, end_date)
opt_collection = opt_collection.filterBounds(roi)
opt_info = opt_collection.getInfo()

In [30]:
for info in opt_info['features']:
    img_date = datetime.datetime.strptime(info['properties']['DATATAKE_IDENTIFIER'][5:13], '%Y%m%d')
    img_lct = info['properties']['PRODUCT_ID']
    cloud = info['properties']['CLOUDY_PIXEL_PERCENTAGE']
    print(img_date, img_lct, cloud)
    #break

2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MXM_20190626T154048 0.1723
2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MXN_20190626T154048 2.4004
2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MYM_20190626T154048 0
2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MYN_20190626T154048 0
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MXM_20190701T154033 0
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MXN_20190701T154033 0.0128
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MYM_20190701T154033 0
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MYN_20190701T154033 0
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R067_T21MXM_20190706T153903 45.7992
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R067_T21MXN_20190706T153903 70.2141
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R067_T21MYM_20190706T153903 14.0007
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R

In [31]:
for date_i in cfg['opt_images_dates']:
    opt_d0 = date_i[0]
    opt_d1 = date_i[1]
    opt_collection_i = opt_collection.filterDate(opt_d0, opt_d1)
    opt_mosaic_i = opt_collection_i.mosaic()
    export_opt_l1(opt_mosaic_i, roi, f'{opt_d0}-{opt_d1}')
    map_view.addLayer(opt_mosaic_i.clip(roi), opt_vizParams, f'opt-{opt_d0}-{opt_d1}')
    print(f'{opt_d0}-{opt_d1}.tif')
    #print(f'{len(opt_collection_i.getInfo()["features"])} images')    

2021-07-25-2021-07-26.tif
2021-08-04-2021-08-05.tif
